In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pprint
import torch.nn as nn

import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv1d, Conv2d,MaxPool1d, MaxPool2d, Module, Softmax, BatchNorm1d, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.nn.functional as F

# torchvision for pre-trained models
from torchvision import models

In [2]:
cwd = os.getcwd() 

In [3]:
master_csv = pd.read_csv(r"C:\Users\sonam\University of Canberra\4th Semester\Capstone\Models\Session_1_labels.csv")
master_csv.head()


,filename,session,subject,emotion
0,cz_eeg_data_1.csv,1,cz,1
1,cz_eeg_data_2.csv,1,cz,2
2,cz_eeg_data_3.csv,1,cz,3
3,cz_eeg_data_4.csv,1,cz,0
4,cz_eeg_data_5.csv,1,cz,2


In [5]:

folder_path = os.path.join(cwd,"session_1_all_200")

eeg_data=[]
for filename in tqdm(master_csv['filename']):
    file_path = os.path.join(folder_path, filename)
    file = pd.read_csv(file_path)
    
    # Drop the first row (column headers)
    file = file.iloc[1:] 
    
    # Drop rows below row 9600
    file = file.iloc[:6002]
    
    file = file.astype('float32')
    eeg_values = file.drop(columns=['time']).values  # Assuming 'time' column is not needed
    
    # Transpose the EEG data
    eeg_values= eeg_values.T
    
#     # Calculate mean and standard deviation for Z-score normalization
#     mean = np.mean(eeg_values, axis=0)
#     std_dev = np.std(eeg_values, axis=0)
    
#     # Apply Z-score normalization
#     eeg_values_normalized = (eeg_values - mean) / std_dev
    
    eeg_data.append(eeg_values)
    
# converting the list to numpy array
eeg_numpy = np.array(eeg_data)
eeg_numpy.shape



100%|████████████████████████████████████████████████████████████████████████████████| 360/360 [05:20<00:00,  1.12it/s]


(360, 62, 6002)

In [107]:
eeg_numpy

array([[[ 1.8298626e+07,  1.3113022e+07,  1.2934208e+07, ...,
         -7.7784060e+06, -1.3977289e+07, -1.7225742e+07],
        [ 1.9699336e+07,  1.3023615e+07,  2.1100044e+07, ...,
         -1.1712313e+07, -1.8119812e+07, -1.1950731e+07],
        [ 1.6123056e+07,  1.2725592e+07,  1.7762184e+07, ...,
         -3.1501054e+07, -3.5375360e+07, -3.0547380e+07],
        ...,
        [ 2.0742416e+07,  2.4825334e+07,  3.6150216e+07, ...,
         -2.5898218e+07, -1.7911196e+07, -1.4126301e+07],
        [ 2.2798776e+07,  2.6345252e+07,  3.4958124e+07, ...,
         -1.4543533e+07, -4.6491625e+06, -9.2387200e+05],
        [ 3.3259392e+07,  3.9786104e+07,  5.0812956e+07, ...,
         -2.3096800e+07, -1.2308359e+07, -1.0937452e+07]],

       [[-3.8146972e+06, -2.1159650e+06, -1.7225742e+07, ...,
         -2.6166440e+07, -3.0547380e+07, -2.3365020e+07],
        [ 1.8477440e+06, -5.0663947e+05, -1.2397766e+07, ...,
         -1.8745660e+07, -2.5302172e+07, -2.2500754e+07],
        [ 8.5234640e+06, 

In [108]:
eeg_numpy.shape

(360, 62, 6002)

In [109]:
# defining the target
label_emotion = master_csv['emotion'].values
print(label_emotion[:48])  # Display the first 10 elements

[1 2 3 0 2 0 0 1 0 1 2 1 1 1 2 3 2 2 3 3 0 3 0 3 1 2 3 0 2 0 0 1 0 1 2 1 1
 1 2 3 2 2 3 3 0 3 0 3]


In [110]:
# create validation set
train_x, val_x, train_y, val_y = train_test_split(eeg_numpy, label_emotion, test_size = 0.25, random_state = 13, stratify=label_emotion)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((270, 62, 6002), (270,)), ((90, 62, 6002), (90,)))

In [111]:
# converting training images into torch format
# Reshape training data
# train_x = train_x.reshape(270, 1, 6002, 62)  # Adding a single channel (assuming grayscale)
train_x = torch.from_numpy(train_x)

In [112]:
# converting the target into torch format
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y)


In [113]:
# shape of training data
train_x.shape, train_y.shape

(torch.Size([270, 62, 6002]), torch.Size([270]))

In [114]:
# Reshape test data
# val_x = val_x.reshape(90, 1, 6002, 62)  # Adding a single channel (assuming grayscale)
val_x = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)


In [115]:
# shape of validation data
val_x.shape, val_y.shape

(torch.Size([90, 62, 6002]), torch.Size([90]))

In [116]:
class Net(Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1d_layers = Sequential(
            Conv1d(62, 4, kernel_size=3, stride=1, padding=1),  
            BatchNorm1d(4),
            ReLU(inplace=True),
            MaxPool1d(kernel_size=2, stride=2),
            Conv1d(4, 8, kernel_size=3, stride=1, padding=1),
            BatchNorm1d(8),
            ReLU(inplace=True),
            MaxPool1d(kernel_size=2, stride=2)
        )

        # Calculate the number of features after convolutional layers
        self.num_features = self._calculate_num_features((62, 6002))

        self.linear_layers = Sequential(
            Linear(self.num_features, 64),
            ReLU(inplace=True),
            Linear(64, 4)
        )

    def _calculate_num_features(self, input_dim):
        input_tensor = torch.zeros(1, *input_dim)
        conv_output = self.conv1d_layers(input_tensor)
        num_features = conv_output.view(1, -1).size(1)
        return num_features

    def forward(self, x):
        x = self.conv1d_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [117]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.0001)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
#if torch.cuda.is_available():
device='cpu'
model = model.to(device)
criterion = criterion.to(device)

print(model)

# batch size of the model
batch_size = 200

# number of epochs to train the model
n_epochs = 30


Net(
  (conv1d_layers): Sequential(
    (0): Conv1d(62, 4, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(4, 8, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=12000, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=4, bias=True)
  )
)


In [118]:
for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
        
    permutation = torch.randperm(train_x.size()[0])

    training_loss = []
    for i in tqdm(range(0,train_x.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        # Convert batch_y to Long data type
        batch_y = batch_y.long()  # Convert to Long data type
        
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        # in case you wanted a semi-full example
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.50it/s]


epoch: 	 1 	 training loss: 	 1.414980173110962


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.59it/s]


epoch: 	 2 	 training loss: 	 1.351163387298584


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.51it/s]


...

epoch: 	 28 	 training loss: 	 0.8137241899967194


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.33it/s]


epoch: 	 29 	 training loss: 	 0.6767475306987762


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.40it/s]

epoch: 	 30 	 training loss: 	 0.7267604470252991


In [119]:
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(train_x.size()[0])
for i in tqdm(range(0,train_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = train_x[indices], train_y[indices]

    #if torch.cuda.is_available():
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    with torch.no_grad():
        output = model(batch_x)

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i],prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.82it/s]

training accuracy: 	 0.8875


In [120]:
# prediction for validation set
prediction_val = []
target_val = []
permutation = torch.randperm(val_x.size()[0])
for i in tqdm(range(0,val_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = val_x[indices], val_y[indices]

    #if torch.cuda.is_available():
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    with torch.no_grad():
        output = model(batch_x)

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction_val.append(predictions)
    target_val.append(batch_y)
    
# validation accuracy
accuracy_val = []
for i in range(len(prediction_val)):
    accuracy_val.append(accuracy_score(target_val[i],prediction_val[i]))
    
print('validation accuracy: \t', np.average(accuracy_val))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.90it/s]

validation accuracy: 	 0.4444444444444444
